In [1]:
import os
import pandas as pd
import shutil
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

OUT_DIR = "/data/yolo_gtsrb"
for split in ["train", "val"]:
    os.makedirs(os.path.join(OUT_DIR, "images", split), exist_ok=True)
    os.makedirs(os.path.join(OUT_DIR, "labels", split), exist_ok=True)

In [19]:
df=pd.read_csv("../data/Balanced_Train_Detection/Train1.csv")
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42
                                    , stratify=df["ClassId"])


In [20]:

def convert_to_yolo(df, split):
    for i, row in tqdm(df.iterrows(), total=len(df)):
        img_path = os.path.join(r"../data/Balanced_Train_Detection", row["Path"])
        label = row["ClassId"]

        # Copy image
        fname = f"{split}_{i}.jpg"
        out_img = os.path.join(OUT_DIR, "images", split, fname)
        shutil.copy(img_path, out_img)

        # Get width, height
        img = cv2.imread(img_path)
        h, w = img.shape[:2]

        # Bounding box = full image
        x_center, y_center = 0.5, 0.5
        width, height = 1.0, 1.0

        # Save label
        out_label = os.path.join(OUT_DIR, "labels", split, fname.replace(".jpg", ".txt"))
        with open(out_label, "w") as f:
            f.write(f"{label} {x_center} {y_center} {width} {height}\n")

convert_to_yolo(train_df, "train")
convert_to_yolo(val_df, "val")


100%|██████████| 17200/17200 [00:34<00:00, 504.66it/s]


In [21]:
classes = {
    0:'Speed limit (20km/h)', 1:'Speed limit (30km/h)', 2:'Speed limit (50km/h)',
    3:'Speed limit (60km/h)', 4:'Speed limit (70km/h)', 5:'Speed limit (80km/h)',
    6:'End of speed limit (80km/h)', 7:'Speed limit (100km/h)', 8:'Speed limit (120km/h)',
    9:'No passing', 10:'No passing veh over 3.5 tons', 11:'Right-of-way at intersection',
    12:'Priority road', 13:'Yield', 14:'Stop', 15:'No vehicles', 16:'Veh > 3.5 tons prohibited',
    17:'No entry', 18:'General caution', 19:'Dangerous curve left', 20:'Dangerous curve right',
    21:'Double curve', 22:'Bumpy road', 23:'Slippery road', 24:'Road narrows on the right',
    25:'Road work', 26:'Traffic signals', 27:'Pedestrians', 28:'Children crossing',
    29:'Bicycles crossing', 30:'Beware of ice/snow', 31:'Wild animals crossing',
    32:'End speed + passing limits', 33:'Turn right ahead', 34:'Turn left ahead',
    35:'Ahead only', 36:'Go straight or right', 37:'Go straight or left', 38:'Keep right',
    39:'Keep left', 40:'Roundabout mandatory', 41:'End of no passing', 42:'End no passing veh > 3.5 tons'
}
with open(os.path.join(OUT_DIR, "data.yaml"), "w") as f:
    f.write(f"""
train: {OUT_DIR}/images/train
val: {OUT_DIR}/images/val
nc: 43
names: {list(classes.values())}
""")


In [ ]:

# Load pretrained YOLOv8 model
model = YOLO("yolov8n.pt")
yaml_path = r"D:\AI-MachineLearning\Ml-project\data\yolo_gtsrb\data.yaml"
print(os.path.exists(yaml_path))  # should print True now

model.train(
    data=yaml_path,
    epochs=2,
    imgsz=64,
    batch=16
)


In [59]:
metrics = model.val()


Ultralytics 8.3.203  Python-3.13.5 torch-2.8.0+cpu CPU (Intel Core i7-8850H 2.60GHz)
Model summary (fused): 72 layers, 3,014,033 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 39.719.3 MB/s, size: 6.4 KB)
val: Scanning D:\AI-MachineLearning\Ml-project\data\yolo_gtsrb\labels\val.cache... 17200 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 17200/17200 11.5Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1075/1075 10.5it/s 1:42<0.1s
                   all      17200      17200      0.275      0.746      0.355      0.339
  Speed limit (20km/h)        400        400       0.28      0.827      0.292      0.277
  Speed limit (30km/h)        400        400      0.198      0.715      0.203      0.188
  Speed limit (50km/h)        400        400      0.256      0.877      0.297      0.294
  Speed limit (60km/h)        400        400      0.183      0.226      0.179      0.177
  Speed 